# Address Segmentation
Conversion of address points into segmented address ranges along a road network.

**Notes:** The following guide assumes data has already been preprocessed including data scrubbing and filtering.

In [1]:
import contextily as ctx
import geopandas as gpd
import ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import shapely
from bisect import bisect
from collections import OrderedDict
from IPython.display import display, HTML
from operator import itemgetter
from shapely.geometry import LineString, Point
HTML('<style>.output {flex-direction: row;}</style>')

## Load dataframes and configure attributes
Loads dataframes into geopandas and separates address numbers and suffixes, if required.

In [2]:
# Load dataframes.
addresses = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="addresses")
roadseg = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="roads")

# Configure attributes - number and suffix.
addresses["suffix"] = addresses["number"].map(lambda val: re.sub(pattern="\\d+", repl="", string=val, flags=re.I))
addresses["number"] = addresses["number"].map(lambda val: re.sub(pattern="[^\\d]", repl="", string=val, flags=re.I)).map(int)

In [3]:
addresses.head()

,number,street,geometry,suffix
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B


In [4]:
roadseg.head()

,featurecod,community,routename,surface,date,roadname,geometry
0,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,VEE LAKE ROAD,"LINESTRING Z (-12731561.088 8988790.623 0.000,..."
1,RRESLIN,Northwest Territories,None,Unpaved,2015-09-28,None,"LINESTRING Z (-12731215.603 8990017.829 0.000,..."
2,RHWYLIN,Northwest Territories,Highway 4,Paved,2015-09-28,INGRAHAM TRAIL,"LINESTRING Z (-12723960.099 8980833.256 0.000,..."
3,RARTLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH ROAD,"LINESTRING Z (-12723909.403 8972227.140 0.000,..."
4,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH,"LINESTRING Z (-12724863.590 8957290.788 0.000,..."


## Preview data

In [5]:
%matplotlib widget

# Fetch basemap.
# Note: basemaps are retrieved in EPSG:3857 and, therefore, dataframes should also use this crs.
basemap, extent = ctx.bounds2img(*roadseg.total_bounds, ll=False, source=ctx.providers.Esri.WorldImagery)

# Configure plot.
fig, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses.plot(ax=ax, color="red", label="addresses", markersize=3)
roadseg.plot(ax=ax, color="cyan", label="roadseg", linewidth=1)
ax.ticklabel_format(useOffset=False, style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.tick_params(labelsize=6)
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("City of Yellowknife")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=8)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Configure address to roadseg linkages
Links addresses to the nearest, matching road segment.

In [6]:
# Link addresses and roadseg on join fields.
addresses["roadseg_index"] = addresses["street"].map(lambda val: tuple(set(roadseg[roadseg["roadname"] == val].index)))
addresses.head()

,number,street,geometry,suffix,roadseg_index
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A,"(389, 390, 391, 361, 911, 370)"
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),,"(1382, 1383, 168, 170, 651, 650, 173, 652, 654..."
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B,"(804, 807)"
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B,"(800, 801, 802, 803, 806, 338)"
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B,"(342,)"


In [7]:
# Reduce linkages to one roadseg index per address.

def get_nearest_linkage(pt, roadseg_indexes):
    """Returns the roadseg index associated with the nearest roadseg geometry to the given address point."""
    
    # Get roadseg geometries.
    roadseg_geometries = tuple(map(lambda index: roadseg["geometry"].iloc[index], roadseg_indexes))
    
    # Get roadseg distances from address point.
    roadseg_distances = tuple(map(lambda road: pt.distance(road), roadseg_geometries))
    
    # Get the roadseg index associated with the smallest distance.
    roadseg_index = roadseg_indexes[roadseg_distances.index(min(roadseg_distances))]
    
    return roadseg_index


# Flag plural linkages.
flag_plural = addresses["roadseg_index"].map(len) > 1

# Reduce plural linkages to the road segment with the lowest (nearest) geometric distance.
addresses.loc[flag_plural, "roadseg_index"] = addresses[flag_plural][["geometry", "roadseg_index"]].apply(
    lambda row: get_nearest_linkage(*row), axis=1)

# Unpack first tuple element for singular linkages.
addresses.loc[~flag_plural, "roadseg_index"] = addresses[~flag_plural]["roadseg_index"].map(itemgetter(0))

# Compile linked roadseg geometry for each address.
addresses["roadseg_geometry"] = addresses.merge(
    roadseg["geometry"], how="left", left_on="roadseg_index", right_index=True)["geometry_y"]

addresses.head()

,number,street,geometry,suffix,roadseg_index,roadseg_geometry
0,43,OTTO DRIVE,POINT Z (-12728491.168 8972072.805 0.000),A,390,"LINESTRING Z (-12728785.212 8972077.288 0.000,..."
1,48,TAYLOR ROAD,POINT Z (-12733829.550 8964622.287 0.000),,168,"LINESTRING Z (-12733834.171 8964493.879 0.000,..."
2,136,DE WEERDT DRIVE,POINT Z (-12731620.348 8969989.275 0.000),B,807,"LINESTRING Z (-12731747.194 8969855.676 0.000,..."
3,129,HAENER DRIVE,POINT Z (-12732186.396 8969861.224 0.000),B,338,"LINESTRING Z (-12732196.969 8969779.397 0.000,..."
4,2,STIRLING COURT,POINT Z (-12731633.895 8970568.762 0.000),B,342,"LINESTRING Z (-12731536.519 8970569.562 0.000,..."


## Configure address parity
Computes address-roadseg parity (left / right side).

In [8]:
def get_parity(pt, vector):
    """
    Determines the parity (left or right side) of an address point relative to a roadseg vector.

    Parity is derived from the determinant of the vectors formed by the road segment and the address-to-roadseg
    vectors. A positive determinant indicates 'left' parity and negative determinant indicates 'right' parity.
    """
    
    det = (vector[1][0] - vector[0][0]) * (pt.y - vector[0][1]) - \
          (vector[1][1] - vector[0][1]) * (pt.x - vector[0][0])
    sign = np.sign(det)
    
    return "l" if sign == 1 else "r"

def get_road_vector(pt, segment):
    """
    Returns the following:
    a) the distance of the address intersection along the road segment.
    b) the vector comprised of the road segment coordinates immediately before and after the address
    intersection point.
    """
    
    # For all road segment points and the intersection point, calculate the distance along the road segment.
    # Note: always use the length as the distance for the last point to avoid distance=0 for looped roads.
    node_distance = (*map(lambda coord: segment.project(Point(coord)), segment.coords[:-1]), segment.length)
    intersection_distance = segment.project(pt)
    
    # Compute the index of the intersection point within the road segment points, based on distances.
    intersection_index = bisect(node_distance, intersection_distance)
    
    # Conditionally compile the road segment points, as a vector, immediately bounding the intersection point.
    
    # Intersection matches a pre-existing road segment point.
    if intersection_distance in node_distance:
        
        # Intersection matches the first road segment point.
        if intersection_index == 1:
            vector = itemgetter(intersection_index - 1, intersection_index)(segment.coords)
        
        # Intersection matches the last road segment point.
        elif intersection_index == len(node_distance):
            vector = itemgetter(intersection_index - 2, intersection_index - 1)(segment.coords)
        
        # Intersection matches an interior road segment point.
        else:
            vector = itemgetter(intersection_index - 2, intersection_index)(segment.coords)
    
    # Intersection matches no pre-existing road segment point.
    else:
        vector = itemgetter(intersection_index - 1, intersection_index)(segment.coords)
    
    return intersection_distance, vector

# Get point of intersection between each address and the linked road segment.
addresses["intersection"] = addresses[["geometry", "roadseg_geometry"]].apply(
    lambda row: itemgetter(-1)(shapely.ops.nearest_points(*row)), axis=1)

# Get the following:
# a) the distance of the intersection point along the linked road segment.
# b) the road segment vector which bounds the intersection point.
#    i.e. vector formed by the coordinates immediately before and after the intersection point.
results = addresses[["intersection", "roadseg_geometry"]].apply(lambda row: get_road_vector(*row), axis=1)
addresses["distance"] = results.map(itemgetter(0))
addresses["roadseg_vector"] = results.map(itemgetter(1))

# Get address parity.
addresses["parity"] = addresses[["geometry", "roadseg_vector"]].apply(
    lambda row: get_parity(*row), axis=1)

addresses[["geometry", "roadseg_geometry", "intersection", "distance", "roadseg_vector", "parity"]].head()

,geometry,roadseg_geometry,intersection,distance,roadseg_vector,parity
0,POINT Z (-12728491.168 8972072.805 0.000),"LINESTRING Z (-12728785.212 8972077.288 0.000,...",POINT (-12728635.1048 8971969.687900001),186.255702,"((-12728647.616799997, 8971984.796699999, 0.0)...",l
1,POINT Z (-12733829.550 8964622.287 0.000),"LINESTRING Z (-12733834.171 8964493.879 0.000,...",POINT (-12733809.82087993 8964497.738158945),24.653609,"((-12733820.3273, 8964496.073899997, 0.0), (-1...",l
2,POINT Z (-12731620.348 8969989.275 0.000),"LINESTRING Z (-12731747.194 8969855.676 0.000,...",POINT (-12731628.7192 8969920.246899998),135.547132,"((-12731657.712499999, 8969910.514499994, 0.0)...",l
3,POINT Z (-12732186.396 8969861.224 0.000),"LINESTRING Z (-12732196.969 8969779.397 0.000,...",POINT (-12732219.69627087 8969849.68197122),73.873501,"((-12732213.9375, 8969833.066699998, 0.0), (-1...",r
4,POINT Z (-12731633.895 8970568.762 0.000),"LINESTRING Z (-12731536.519 8970569.562 0.000,...",POINT (-12731619.42874114 8970534.401334839),90.057500,"((-12731587.3267, 8970547.916899998, 0.0), (-1...",r


## View relationship between parity inputs
View the relationship between address points, bounding roadseg vectors, and address-roadseg intersection points.

In [9]:
# Fetch basemap.
basemap, extent = ctx.bounds2img(*addresses.total_bounds, ll=False, source=ctx.providers.Esri.WorldImagery)

# Create geometries for viewing.
bounding_roadseg_vectors = gpd.GeoDataFrame(geometry=addresses["roadseg_vector"].map(LineString), crs=addresses.crs)
intersection_pt_linkages = gpd.GeoDataFrame(geometry=addresses[["geometry", "intersection"]].apply(
    lambda row: LineString([pt.coords[0][:2] for pt in row]), axis=1), crs=addresses.crs)

# Configure plot.
fix, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses.plot(ax=ax, color="red", label="addresses", markersize=3)
intersection_pt_linkages.plot(ax=ax, color="yellow", label="addresses - intersection point linkages", linewidth=1)
bounding_roadseg_vectors.plot(ax=ax, color="cyan", label="bounding roadseg vectors", linewidth=1)
ax.ticklabel_format(useOffset=False, style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.tick_params(labelsize=6)
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("City of Yellowknife")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=8)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Configure address ranges (addrange) and attributes
Groups addresses into ranges by roadseg linkage and parity, then computes the range attributes.

In [10]:
def get_digdirfg(sequence):
    """Returns the digdirfg attribute for the given sequence of address numbers."""

    sequence = list(sequence)

    # Return digitizing direction for single addresses.
    if len(sequence) == 1:
        return "Not Applicable"

    # Derive digitizing direction from sequence sorting direction.
    if sequence == sorted(sequence):
        return "Same Direction"
    else:
        return "Opposite Direction"

def get_hnumstr(sequence):
    """Returns the hnumstr attribute for the given sequence of address numbers."""

    sequence = list(sequence)

    # Validate structure for single addresses.
    if len(sequence) == 1:
        return "Even" if (sequence[0] % 2 == 0) else "Odd"

    # Configure sequence sort status.
    if sequence == sorted(sequence) or sequence == sorted(sequence, reverse=True):

        # Configure sequence parities.
        parities = tuple(map(lambda number: number % 2 == 0, sequence))

        # Validate structure for sorted address ranges.
        if all(parities):
            return "Even"
        elif not any(parities):
            return "Odd"
        else:
            return "Mixed"

    # Return structure for unsorted address ranges.
    else:
        return "Irregular"

def get_number_sequence(addresses):
    """Returns the filtered number sequence for the given addresses."""

    # Separate address components.
    numbers, suffixes, distances = tuple(zip(*addresses))

    # Reduce addresses at a duplicated intersection distance to only the first instance.
    if len(distances) == len(set(distances)):
        sequence = numbers
    else:
        sequence = pd.DataFrame({"number": numbers, "suffix": suffixes, "distance": distances}).drop_duplicates(
            subset="distance", keep="first")["number"].to_list()

    # Remove duplicated addresses.
    sequence = list(OrderedDict.fromkeys(sequence))

    return sequence

def groupby_to_list(df, group_field, list_field):
    """
    Helper function: faster alternative to pandas groupby.apply/agg(list).
    Groups records by one or more fields and compiles an output field into a list for each group.
    """
    
    if isinstance(group_field, list):
        for field in group_field:
            if df[field].dtype.name != "geometry":
                df[field] = df[field].astype("U")
        transpose = df.sort_values(group_field)[[*group_field, list_field]].values.T
        keys, vals = np.column_stack(transpose[:-1]), transpose[-1]
        keys_unique, keys_indexes = np.unique(keys.astype("U") if isinstance(keys, np.object) else keys, 
                                              axis=0, return_index=True)
    
    else:
        keys, vals = df.sort_values(group_field)[[group_field, list_field]].values.T
        keys_unique, keys_indexes = np.unique(keys, return_index=True)
    
    vals_arrays = np.split(vals, keys_indexes[1:])
    
    return pd.Series([list(vals_array) for vals_array in vals_arrays], index=keys_unique).copy(deep=True)

def sort_addresses(numbers, suffixes, distances):
    """
    Sorts the addresses successively by:
    1) distance - the distance of the intersection point along the road segment.
    2) number
    3) suffix
    Taking into account the directionality of the addresses relative to the road segment.
    """

    # Create individual address tuples from separated address components.
    addresses = tuple(zip(numbers, suffixes, distances))

    # Apply initial sorting, by distance, to identify address directionality.
    addresses_sorted = sorted(addresses, key=itemgetter(2))
    directionality = -1 if addresses_sorted[0][0] > addresses_sorted[-1][0] else 1

    # Sort addresses - same direction.
    if directionality == 1:
        return tuple(sorted(addresses, key=itemgetter(2, 1, 0)))

    # Sort addresses - opposite direction.
    else:
        return tuple(sorted(sorted(sorted(
            addresses, key=itemgetter(1), reverse=True),
            key=itemgetter(0), reverse=True),
            key=itemgetter(2)))

In [11]:
# Split address dataframe by parity.
addresses_l = addresses[addresses["parity"] == "l"].copy(deep=True)
addresses_r = addresses[addresses["parity"] == "r"].copy(deep=True)

# Create dataframes from grouped addresses.
cols = ("number", "suffix", "distance")
addresses_l = pd.DataFrame({col: groupby_to_list(addresses_l, "roadseg_index", col) for col in cols})
addresses_r = pd.DataFrame({col: groupby_to_list(addresses_r, "roadseg_index", col) for col in cols})

# Sort addresses.
addresses_l = addresses_l.apply(lambda row: sort_addresses(*row), axis=1)
addresses_r = addresses_r.apply(lambda row: sort_addresses(*row), axis=1)

**Example of a single address grouping, sorted (index=264):**

In [12]:
# View example address group.
vals_l = list(zip(*addresses_l.loc[addresses_l.index==264].iloc[0]))
vals_r = list(zip(*addresses_r.loc[addresses_r.index==264].iloc[0]))
display(pd.DataFrame({('Left parity', col): itemgetter(index)(vals_l) for index, col in enumerate(["number", "suffix", "distance"])}))
display(pd.DataFrame({('Right parity', col): itemgetter(index)(vals_r) for index, col in enumerate(["number", "suffix", "distance"])}))

Left parity                   
        number suffix    distance
0          337         106.060976
1          339         233.806352
2          341      B  320.034574
3          341      C  324.614422
4          341      D  324.840869
5          341      E  326.153381
6          341      A  387.711231
7          343         479.407658
8          345         579.482713
9          347         711.903999
10         349         847.309281

Right parity                   
        number suffix    distance
0          338         127.031001
1          340         241.058171
2          342         320.206931
3          344         379.945207
4          346         498.629774
5          348         576.121571
6          350         755.735122
7          352         837.968201

In [13]:
# Configure addrange attributes.
addrange = pd.DataFrame(index=map(int, {*addresses_l.index, *addresses_r.index}))

# Configure addrange attributes - hnumf, hnuml.
addrange.loc[addresses_l.index, "l_hnumf"] = addresses_l.map(lambda addresses: addresses[0][0])
addrange.loc[addresses_l.index, "l_hnuml"] = addresses_l.map(lambda addresses: addresses[-1][0])
addrange.loc[addresses_r.index, "r_hnumf"] = addresses_r.map(lambda addresses: addresses[0][0])
addrange.loc[addresses_r.index, "r_hnuml"] = addresses_r.map(lambda addresses: addresses[-1][0])

# Configuring addrange attributes - hnumsuff, hnumsufl.
addrange.loc[addresses_l.index, "l_hnumsuff"] = addresses_l.map(lambda addresses: addresses[0][1])
addrange.loc[addresses_l.index, "l_hnumsufl"] = addresses_l.map(lambda addresses: addresses[-1][1])
addrange.loc[addresses_r.index, "r_hnumsuff"] = addresses_r.map(lambda addresses: addresses[0][1])
addrange.loc[addresses_r.index, "r_hnumsufl"] = addresses_r.map(lambda addresses: addresses[-1][1])

# Configuring addrange attributes - hnumtypf, hnumtypl.
addrange.loc[addresses_l.index, "l_hnumtypf"] = addresses_l.map(lambda addresses: "Actual Located")
addrange.loc[addresses_l.index, "l_hnumtypl"] = addresses_l.map(lambda addresses: "Actual Located")
addrange.loc[addresses_r.index, "r_hnumtypf"] = addresses_r.map(lambda addresses: "Actual Located")
addrange.loc[addresses_r.index, "r_hnumtypl"] = addresses_r.map(lambda addresses: "Actual Located")

# Get address number sequence.
address_sequence_l = addresses_l.map(get_number_sequence)
address_sequence_r = addresses_r.map(get_number_sequence)

# Configure addrange attributes - hnumstr.
addrange.loc[addresses_l.index, "l_hnumstr"] = address_sequence_l.map(get_hnumstr)
addrange.loc[addresses_r.index, "r_hnumstr"] = address_sequence_r.map(get_hnumstr)

# Configure addrange attributes - digdirfg.
addrange.loc[addresses_l.index, "l_digdirfg"] = address_sequence_l.map(get_digdirfg)
addrange.loc[addresses_r.index, "r_digdirfg"] = address_sequence_r.map(get_digdirfg)

**Example addrange attributes (index=264):**

In [14]:
# View example address group.
addrange.loc[addrange.index==264].iloc[0]

l_hnumf                  337
l_hnuml                  349
r_hnumf                  338
r_hnuml                  352
l_hnumsuff                  
l_hnumsufl                  
r_hnumsuff                  
r_hnumsufl                  
l_hnumtypf    Actual Located
l_hnumtypl    Actual Located
r_hnumtypf    Actual Located
r_hnumtypl    Actual Located
l_hnumstr                Odd
r_hnumstr               Even
l_digdirfg    Same Direction
r_digdirfg    Same Direction
Name: 264, dtype: object

## Merge addrange attributes with roadseg

In [15]:
# Merge addrange attributes with roadseg.
roadseg = roadseg.merge(addrange, how="left", left_index=True, right_index=True)

**Example roadseg - addrange merge (index=264):**

In [16]:
# View example address group.
roadseg.loc[roadseg.index==264].iloc[0]

featurecod                                              RARTLIN
community                                           Yellowknife
routename                                                  None
surface                                                   Paved
date                                                 2015-09-28
roadname                                       OLD AIRPORT ROAD
geometry      LINESTRING Z (-12735876.4681 8967934.843400002...
l_hnumf                                                     337
l_hnuml                                                     349
r_hnumf                                                     338
r_hnuml                                                     352
l_hnumsuff                                                     
l_hnumsufl                                                     
r_hnumsuff                                                     
r_hnumsufl                                                     
l_hnumtypf                              

In [17]:
# Create data for viewing.
intersection_pt_linkages = gpd.GeoDataFrame(
    geometry=addresses.loc[addresses["roadseg_index"]==264][["geometry", "intersection"]].apply(
        lambda row: LineString([pt.coords[0][:2] for pt in row]), axis=1), crs=addresses.crs)
addresses_filtered = addresses.loc[addresses["roadseg_index"]==264]
labels = addresses_filtered[["number", "suffix", "geometry", "parity"]].apply(
    lambda row: (f"{row[0]}{row[1]}", row[2].x, row[2].y, row[3]), axis=1)

# Fetch basemap.
basemap, extent = ctx.bounds2img(*addresses_filtered.total_bounds, ll=False, source=ctx.providers.Esri.WorldImagery)

# Configure plot.
fix, ax = plt.subplots(tight_layout=True)
plt.imshow(basemap, extent=extent)
addresses_filtered.plot(ax=ax, color="red", label="addresses", markersize=10)
intersection_pt_linkages.plot(ax=ax, color="yellow", label="addresses - intersection point linkages", linewidth=1)
roadseg.loc[roadseg.index==264].plot(ax=ax, color="cyan", label="roadseg", linewidth=1)
ax.ticklabel_format(useOffset=False, style="plain")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha="right")
plt.tick_params(labelsize=6)
plt.xlabel("Longitude (m)")
plt.ylabel("Latitude (m)")
plt.title("City of Yellowknife\nExample address ranges (index=264)")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=8)
for label_params in labels:
    label, x, y, parity = label_params
    if parity == "l":
        plt.annotate(label, xy=(x, y), xytext=(x-20, y-10), textcoords="data", ha="right", va="top", fontsize=6, color="red", fontweight="bold", bbox=dict(pad=0.3, fc="black"))
    else:
        plt.annotate(label, xy=(x, y), xytext=(x+20, y-10), textcoords="data", ha="left", va="top", fontsize=6, color="red", fontweight="bold", bbox=dict(pad=0.3, fc="black"))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …